In [2]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()g standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

## NOTE (1) 
Run the piece of code bellow to: <br>
-> produce a list of characters out of a string <br>
-> produce all possible results of binary combinations

In [3]:
# Code imported form 
# https://stackoverflow.com/questions/14931769/how-to-get-all-combination-of-n-binary-value
import itertools
def binaryCombinations(N): 
    lst = list(itertools.product([0, 1], repeat=N))
    return lst

## NOTE (2)
### Code explanation about **storing measurements** : 

~~~ 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run( circuit_name , shots = N, memory = True )
    output = experiment.result().get_memory()[ i ] 
    
    # usually with N = 1, i = 0
~~~

<br>First, by
~~~ 
    simulation = Aer.get_backend('aer_simulator')
~~~
we connect our backend to the Aer simulator in Qiskit, in order to simulate quantum circuits. <br>
Then, with the command
~~~ 
    experiment = simulation.run( circuit_name , shots = N, memory = True )
~~~
we are executing N experiments running our circuit. It is important to set as True the last parameter, refering to memory. This creates an array of N positions, each one of them saving an experiment execution. <br>
Lastly, we use the command 
~~~ 
    output = experiment.result().get_memory()[ i ] 
~~~
to access measurement result of execution No. (i+1), which is stored in the i-th position of the (zero-based) array. 

In [20]:
def XNOR(inputs): 
    # 0. GATE FUNCTIONALITY / LOGIC
    # If the number of inputs that are '1' is odd, it returns '0'
    # If the number of inputs that are '1' is even, it returns '1'
    # _________________________________
    
    # 1. CREATE QUANTUM CIRCUIT
    # XOR gate needs at least 2 qubits as input line. 
    # More are acceptable. 
    # It also needs a classical bit to store measurement result.
    XNOR_gate = QuantumCircuit(len(inputs),1)
    
    # 2. INTIALIZE CIRCUIT TO MATCH FUNCTION INPUT
    # Due to Qiskit initializing qubits to |0>, 
    # we need to add an X gate to each input that is '1'
    for i in range(len(inputs)):
        if (inputs[i] == 1):
            XNOR_gate.x(i)
    XNOR_gate.barrier()
    
    # 3. ADD GATES TO ACHIEVE DESIRED OUTPUT
    for i in range(len(inputs)-1): 
        XNOR_gate.cx(i,i+1)
    XNOR_gate.x(len(inputs)-1)
    XNOR_gate.measure(len(inputs)-1, 0)
    XNOR_gate.draw()
    
    # 4. SIMULATE MEASUREMENT AND RETURN RESULTS 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run(XNOR_gate, shots=1, memory=True)
    output = experiment.result().get_memory()[0]
    
    return XNOR_gate, int(output)

In [5]:
# Executing all possible results for XNOR gate
# using N qubits as input
N = 3
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = XNOR(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER XNOR gate: "',output_value, '"')
    display(output_qc.draw('text'))
    print('\n')

BEFORE:  " (0, 0, 0) " -->  AFTER XNOR gate: " 1 "


░                   
q_0: ─░───■───────────────
      ░ ┌─┴─┐             
q_1: ─░─┤ X ├──■──────────
      ░ └───┘┌─┴─┐┌───┐┌─┐
q_2: ─░──────┤ X ├┤ X ├┤M├
      ░      └───┘└───┘└╥┘
c: 1/═══════════════════╩═
                        0



BEFORE:  " (0, 0, 1) " -->  AFTER XNOR gate: " 0 "


░                   
q_0: ──────░───■───────────────
           ░ ┌─┴─┐             
q_1: ──────░─┤ X ├──■──────────
     ┌───┐ ░ └───┘┌─┴─┐┌───┐┌─┐
q_2: ┤ X ├─░──────┤ X ├┤ X ├┤M├
     └───┘ ░      └───┘└───┘└╥┘
c: 1/════════════════════════╩═
                             0



BEFORE:  " (0, 1, 0) " -->  AFTER XNOR gate: " 0 "


...

┌───┐ ░                   
q_0: ┤ X ├─░───■───────────────
     └───┘ ░ ┌─┴─┐             
q_1: ──────░─┤ X ├──■──────────
     ┌───┐ ░ └───┘┌─┴─┐┌───┐┌─┐
q_2: ┤ X ├─░──────┤ X ├┤ X ├┤M├
     └───┘ ░      └───┘└───┘└╥┘
c: 1/════════════════════════╩═
                             0



BEFORE:  " (1, 1, 0) " -->  AFTER XNOR gate: " 1 "


┌───┐ ░                   
q_0: ┤ X ├─░───■───────────────
     ├───┤ ░ ┌─┴─┐             
q_1: ┤ X ├─░─┤ X ├──■──────────
     └───┘ ░ └───┘┌─┴─┐┌───┐┌─┐
q_2: ──────░──────┤ X ├┤ X ├┤M├
           ░      └───┘└───┘└╥┘
c: 1/════════════════════════╩═
                             0



BEFORE:  " (1, 1, 1) " -->  AFTER XNOR gate: " 0 "


┌───┐ ░                   
q_0: ┤ X ├─░───■───────────────
     ├───┤ ░ ┌─┴─┐             
q_1: ┤ X ├─░─┤ X ├──■──────────
     ├───┤ ░ └───┘┌─┴─┐┌───┐┌─┐
q_2: ┤ X ├─░──────┤ X ├┤ X ├┤M├
     └───┘ ░      └───┘└───┘└╥┘
c: 1/════════════════════════╩═
                             0

In [9]:
# Executing all possible results for XNOR gate
# using N qubits as input
N = 5
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = XNOR(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER XNOR gate: "',output_value, '"')
    #display(output_qc.draw('text'))
    #print('\n')

BEFORE:  " (0, 0, 0, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 1, 0, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 0, 1, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 1, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 1, 1, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 0, 0, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 0, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 1, 0, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 1, 0, 1, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 1, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 1, 1, 0, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 1, 1, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 1, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (1, 0, 0, 0, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (1, 0, 0, 0, 1) " --

In [24]:
# SAME testing code, but for bigger inputs ( N > 7 ) 
# An execution for N >= 8 require more than half a minute to finish
# For even greater number of inputs, it takes even more. 

N = 8
combinations = binaryCombinations(N)

print("For N = ", N, ", show only ", N, "-bit-combinations that have XNOR output as 1")
for comb in combinations: 
    init_value = comb
    output_qc, output_value = XNOR(init_value)
    if (output_value == 1):
        print('BEFORE:  "',init_value,'" -->  AFTER XNOR gate: "',output_value, '"')
        #display(output_qc.draw('text'))
        #print('\n')

For N =  8 , show only  8 -bit-combinations that have XNOR output as 1
BEFORE:  " (0, 0, 0, 0, 0, 0, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 0, 0, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 0, 1, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 0, 1, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 0, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 0, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 1, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 1, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 0, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 0, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 1, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 1, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 1, 0, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 1, 0, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFOR

In [21]:
# -----------------------------------------------
# ANOTHER IMPLEMENTANTION OF XNOR
# -----------------------------------------------
# USING THE XOR GATE AND INVERTING EACH RESULT 
# LETS NAME IT AS "XNOR_using_XOR(inputs)"
# AND ALSO COPY-PASTE THE "XOR(inputs)" GATE ABOVE. 


def XOR(inputs): 
    # 0. GATE FUNCTIONALITY / LOGIC
    # If the number of inputs that are '1' is odd, it returns '1'
    # If the number of inputs that are '1' is even, it returns '0'
    # _________________________________
    
    # 1. CREATE QUANTUM CIRCUIT
    # XOR gate needs at least 2 qubits as input line. 
    # More are acceptable. 
    # It also needs a classical bit to store measurement result.
    XOR_gate = QuantumCircuit(len(inputs),1)
    
    # 2. INTIALIZE CIRCUIT TO MATCH FUNCTION INPUT
    # Due to Qiskit initializing qubits to |0>, 
    # we need to add an X gate to each input that is '1'
    for i in range(len(inputs)):
        if (inputs[i] == 1):
            XOR_gate.x(i)
    XOR_gate.barrier()
    
    # 3. ADD GATES TO ACHIEVE DESIRED OUTPUT
    for i in range(len(inputs)-1): 
        XOR_gate.cx(i,i+1)
    XOR_gate.measure(len(inputs)-1, 0)
    XOR_gate.draw()
    
    # 4. SIMULATE MEASUREMENT AND RETURN RESULTS 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run(XOR_gate, shots=1, memory=True)
    output = experiment.result().get_memory()[0]
    
    return XOR_gate, int(output)


def XNOR_using_XOR(inputs): 
    # 0. GATE FUNCTIONALITY / LOGIC
    # If the number of inputs that are '1' is odd, it returns '0'
    # If the number of inputs that are '1' is even, it returns '1'
    # _________________________________
    
    # 1. CREATE QUANTUM CIRCUIT
    # XOR gate needs at least 2 qubits as input line. 
    # More are acceptable. 
    # It also needs a classical bit to store measurement result.
    XNOR_gate = QuantumCircuit(len(inputs),1)
    
    # 3. ADD GATES TO ACHIEVE DESIRED OUTPUT
    XOR_gate, XOR_result = XOR(inputs)
    XOR_gate.x(len(inputs)-1)
    XOR_gate.measure(len(inputs)-1, 0)
    XOR_gate.draw()
    
    # 4. SIMULATE MEASUREMENT AND RETURN RESULTS 
    simulation = Aer.get_backend('aer_simulator')
    experiment = simulation.run(XOR_gate, shots=1, memory=True)
    output = experiment.result().get_memory()[0]
    
    return XOR_gate, int(output)

In [14]:
# Executing all possible results for XNOR gate
# using N qubits as input
N = 3
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = XNOR_using_XOR(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER XNOR gate: "',output_value, '"')
    display(output_qc.draw('text'))
    print('\n')

BEFORE:  " (0, 0, 0) " -->  AFTER XNOR gate: " 1 "


░                      
q_0: ─░───■──────────────────
      ░ ┌─┴─┐                
q_1: ─░─┤ X ├──■─────────────
      ░ └───┘┌─┴─┐┌─┐┌───┐┌─┐
q_2: ─░──────┤ X ├┤M├┤ X ├┤M├
      ░      └───┘└╥┘└───┘└╥┘
c: 1/══════════════╩═══════╩═
                   0       0



BEFORE:  " (0, 0, 1) " -->  AFTER XNOR gate: " 0 "


░                      
q_0: ──────░───■──────────────────
           ░ ┌─┴─┐                
q_1: ──────░─┤ X ├──■─────────────
     ┌───┐ ░ └───┘┌─┴─┐┌─┐┌───┐┌─┐
q_2: ┤ X ├─░──────┤ X ├┤M├┤ X ├┤M├
     └───┘ ░      └───┘└╥┘└───┘└╥┘
c: 1/═══════════════════╩═══════╩═
                        0       0



BEFORE:  " (0, 1, 0) " -->  AFTER XNOR gate: " 0 "


...

┌───┐ ░                      
q_0: ┤ X ├─░───■──────────────────
     └───┘ ░ ┌─┴─┐                
q_1: ──────░─┤ X ├──■─────────────
     ┌───┐ ░ └───┘┌─┴─┐┌─┐┌───┐┌─┐
q_2: ┤ X ├─░──────┤ X ├┤M├┤ X ├┤M├
     └───┘ ░      └───┘└╥┘└───┘└╥┘
c: 1/═══════════════════╩═══════╩═
                        0       0



BEFORE:  " (1, 1, 0) " -->  AFTER XNOR gate: " 1 "


┌───┐ ░                      
q_0: ┤ X ├─░───■──────────────────
     ├───┤ ░ ┌─┴─┐                
q_1: ┤ X ├─░─┤ X ├──■─────────────
     └───┘ ░ └───┘┌─┴─┐┌─┐┌───┐┌─┐
q_2: ──────░──────┤ X ├┤M├┤ X ├┤M├
           ░      └───┘└╥┘└───┘└╥┘
c: 1/═══════════════════╩═══════╩═
                        0       0



BEFORE:  " (1, 1, 1) " -->  AFTER XNOR gate: " 0 "


┌───┐ ░                      
q_0: ┤ X ├─░───■──────────────────
     ├───┤ ░ ┌─┴─┐                
q_1: ┤ X ├─░─┤ X ├──■─────────────
     ├───┤ ░ └───┘┌─┴─┐┌─┐┌───┐┌─┐
q_2: ┤ X ├─░──────┤ X ├┤M├┤ X ├┤M├
     └───┘ ░      └───┘└╥┘└───┘└╥┘
c: 1/═══════════════════╩═══════╩═
                        0       0

In [15]:
# Executing all possible results for XNOR gate
# using N qubits as input
N = 5
combinations = binaryCombinations(N)

for comb in combinations: 
    init_value = comb
    output_qc, output_value = XNOR_using_XOR(init_value)
    print('BEFORE:  "',init_value,'" -->  AFTER XNOR gate: "',output_value, '"')
    #display(output_qc.draw('text'))
    #print('\n')

BEFORE:  " (0, 0, 0, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 0, 0, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 1, 0, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 0, 1, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 1, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 1, 1, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 0, 0, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 0, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 1, 0, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 1, 0, 1, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 1, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 1, 1, 0, 1) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 1, 1, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (0, 1, 1, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (1, 0, 0, 0, 0) " -->  AFTER XNOR gate: " 0 "
BEFORE:  " (1, 0, 0, 0, 1) " --

In [25]:
# SAME testing code, but for bigger inputs ( N > 7 ) 
# An execution for N >= 8 require more than half a minute to finish
# For even greater number of inputs, it takes even more. 

N = 8
combinations = binaryCombinations(N)

print("For N = ", N, ", show only ", N, "-bit-combinations that have XNOR output as 1")
for comb in combinations: 
    init_value = comb
    output_qc, output_value = XNOR_using_XOR(init_value)
    if (output_value == 1):
        print('BEFORE:  "',init_value,'" -->  AFTER XNOR gate: "',output_value, '"')
        #display(output_qc.draw('text'))
        #print('\n')

For N =  8 , show only  8 -bit-combinations that have XNOR output as 1
BEFORE:  " (0, 0, 0, 0, 0, 0, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 0, 0, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 0, 1, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 0, 1, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 0, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 0, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 1, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 0, 1, 1, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 0, 0, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 0, 1, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 1, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 0, 1, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 1, 0, 0, 0) " -->  AFTER XNOR gate: " 1 "
BEFORE:  " (0, 0, 0, 1, 1, 0, 1, 1) " -->  AFTER XNOR gate: " 1 "
BEFOR